In [0]:
# Change directory to VSCode workspace root so that relative path loads work correctly. Turn this addition off with the DataScience.changeDirOnImportExport setting
# ms-python.python added
import os
try:
	os.chdir(os.path.join(os.getcwd(), '..'))
	print(os.getcwd())
except:
	pass


 # Demo準備

In [1]:
%%bash
sudo docker rm -f sql19rc1
sudo docker rm -f sql17cu16
sudo docker rm -f registry
sudo docker rmi localhost:5000/mssql/devdb:v1
sudo rm -r ~/SQL_Server_Container/DataVolume/*
sudo rm -r ~/SQL_Server_Container/Dockerfile
echo Cleanup


sql17cu16
Cleanup
Error: No such container: sql19rc1
Error: No such container: registry
Error: No such image: localhost:5000/mssql/devdb:v1
rm: cannot remove '/home/o9o9/SQL_Server_Container/Dockerfile': No such file or directory


 # Demo 1: SQL Server コンテナの実行
 ## 1-1.SQL Server 2017イメージをPull（事前にPull済み）

In [2]:
%%bash
sudo docker pull mcr.microsoft.com/mssql/server:2017-CU16
sudo docker image list | grep 2017-CU16


2017-CU16: Pulling from mssql/server
Digest: sha256:e235aaa983f2ab339de6b7846e93080ad0351cd553cd32d31d5232ccef3a7cbf
Status: Image is up to date for mcr.microsoft.com/mssql/server:2017-CU16
mcr.microsoft.com/mssql/server:2017-CU16
mcr.microsoft.com/mssql/server   2017-CU16            d9b9b96627b7        2 months ago        1.36GB


 ## 1-2.SQL Server 2017コンテナ起動

In [3]:
%%bash
sudo docker run -e 'ACCEPT_EULA=Y' -e 'MSSQL_SA_PASSWORD=P@ssw0rd' -p 11433:1433 -v ~/SQL_Server_Container/DataVolume:/var/opt/mssql --name sql17cu16 -d mcr.microsoft.com/mssql/server:2017-CU16
sudo docker ps -a --filter name=sql17cu16
sleep 30


4deca4493a9071c9d38710cb7149fde9c9e0042325ea8ecc71b62661ce087330
CONTAINER ID        IMAGE                                      COMMAND                  CREATED             STATUS                  PORTS                     NAMES
4deca4493a90        mcr.microsoft.com/mssql/server:2017-CU16   "/opt/mssql/bin/sqls…"   2 seconds ago       Up Less than a second   0.0.0.0:11433->1433/tcp   sql17cu16


 ## 1-3.コンテナ内でsqlcmdを実行（コンテナ内は1433ポートでリッスン）

In [4]:
%%bash
sudo docker exec sql17cu16 /opt/mssql-tools/bin/sqlcmd -S localhost,1433 -U sa -P P@ssw0rd -Q 'select @@version'



------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Microsoft SQL Server 2017 (RTM-CU16) (KB4508218) - 14.0.3223.3 (X64) 
	Jul 12 2019 17:43:08 
	Copyright (C) 2017 Microsoft Corporation
	Developer Edition (64-bit) on Linux (Ubuntu 16.04.6 LTS)                                                                                                            

(1 rows affected)


 ## 1-4.コンテナ外でsqlcmd（ホストの11433ポートはコンテナの1433にマッピング）

In [5]:
%%bash
sqlcmd -S localhost,11433 -U sa -P P@ssw0rd -Q 'select @@version'



------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Microsoft SQL Server 2017 (RTM-CU16) (KB4508218) - 14.0.3223.3 (X64) 
	Jul 12 2019 17:43:08 
	Copyright (C) 2017 Microsoft Corporation
	Developer Edition (64-bit) on Linux (Ubuntu 16.04.6 LTS)                                                                                                            

(1 rows affected)


 ## 1-5.DevDB作成

In [6]:
%%bash
sqlcmd -S localhost,11433 -U sa -P P@ssw0rd -Q 'create database DevDB'
sqlcmd -S localhost,11433 -U sa -P P@ssw0rd -Q 'use DevDB; create table tab01(id int, name varchar(max)); insert into tab01 values (1, "SQL Server runs on Docker!")'
sqlcmd -S localhost,11433 -U sa -P P@ssw0rd -Q 'select name from sys.databases; use DevDB; select * from tab01'


Changed database context to 'DevDB'.

(1 rows affected)
name                                                                                                                            
--------------------------------------------------------------------------------------------------------------------------------
master                                                                                                                          
tempdb                                                                                                                          
model                                                                                                                           
msdb                                                                                                                            
DevDB                                                                                                                           

(5 rows affected)
Changed database conte

 ## 1-6.コンテナ内にDBファイルが作成された

In [9]:
%%bash
sudo docker exec sql17cu16 ls -la /var/opt/mssql/data | grep DevDB


-rw-r----- 1 root root  8388608 Oct 10 12:31 DevDB.mdf
-rw-r----- 1 root root  8388608 Oct 10 12:31 DevDB_log.ldf


 ## 1-7.ホストボリュームからもDBファイルが確認できる

In [10]:
%%bash
ls -la ~/SQL_Server_Container/DataVolume/data | grep DevDB


-rw-r----- 1 root root  8388608 Oct 10 12:31 DevDB_log.ldf
-rw-r----- 1 root root  8388608 Oct 10 12:31 DevDB.mdf


 ## 1-8.コンテナ停止してもホストボリューム上のDBファイルは消えない

In [12]:
%%bash
sudo docker stop sql17cu16
sudo docker ps -a --filter name=sql17cu16
ls -la ~/SQL_Server_Container/DataVolume/data | grep DevDB


sql17cu16
CONTAINER ID        IMAGE                                      COMMAND                  CREATED             STATUS                      PORTS               NAMES
4deca4493a90        mcr.microsoft.com/mssql/server:2017-CU16   "/opt/mssql/bin/sqls…"   2 minutes ago       Exited (0) 14 seconds ago                       sql17cu16
-rw-r----- 1 root root  8388608 Oct 10 12:32 DevDB_log.ldf
-rw-r----- 1 root root  8388608 Oct 10 12:32 DevDB.mdf


 ## 1-9.コンテナ破棄してもホストボリューム上のDBファイルは消えない

In [13]:
%%bash
sudo docker rm -f sql17cu16
sudo docker ps -a --filter name=sql17cu16
ls -la ~/SQL_Server_Container/DataVolume/data | grep DevDB


sql17cu16
CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES
-rw-r----- 1 root root  8388608 Oct 10 12:32 DevDB_log.ldf
-rw-r----- 1 root root  8388608 Oct 10 12:32 DevDB.mdf


 # Demo 2: Docker コンテナ内での操作
 ## 2-1.SQL Server 2017コンテナ起動

In [14]:
%%bash
sudo docker run -e 'ACCEPT_EULA=Y' -e 'MSSQL_SA_PASSWORD=P@ssw0rd' -p 11433:1433 -v ~/SQL_Server_Container/DataVolume:/var/opt/mssql --name sql17cu16 -d mcr.microsoft.com/mssql/server:2017-CU16
sudo docker ps -a --filter name=sql17cu16
sleep 30


40b0acfaadb380f08b6fc71376618c87d49b856fcd4b1c7c9cf44693207b51e6
CONTAINER ID        IMAGE                                      COMMAND                  CREATED             STATUS                  PORTS                     NAMES
40b0acfaadb3        mcr.microsoft.com/mssql/server:2017-CU16   "/opt/mssql/bin/sqls…"   1 second ago        Up Less than a second   0.0.0.0:11433->1433/tcp   sql17cu16


 ## 2-2.コンテナ接続
 sudo docker exec -it sql17cu16 /bin/bash
 ## 2-3.コンテナ内でmssql-cliインストール
 > apt-get update
 > apt-get install mssql-cli
 ![](image/2019-10-08-23-26-04.png)
 > which mssql-cli
 > mssql-cli -S localhost -U sa -P P@ssw0rd
 ![](image/2019-10-08-23-29-40.png)

 ## 2-4.ホスト側にはmssql-cliは存在しない
 /usr/bin/mssql-cli
 ![](image/2019-10-08-23-36-07.png)

 ## 2-5.コンテナ破棄

In [15]:
%%bash
sudo docker rm -f sql17cu16
sudo docker ps -a --filter name=sql17cu16


sql17cu16
CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES


 # Demo 3: SQL Server コンテナ内のUpgrade（2017->2019）
 ## 3-1.SQL Server 2019イメージをPull（事前にPull済み）

In [16]:
%%bash
sudo docker pull mcr.microsoft.com/mssql/server:2019-RC1
sudo docker image list | grep 2019-RC1


2019-RC1: Pulling from mssql/server
Digest: sha256:a11facbda1b1cc299d4a37499ef79cd18e38bfb8e5dd7e45cc73670cc07772e5
Status: Image is up to date for mcr.microsoft.com/mssql/server:2019-RC1
mcr.microsoft.com/mssql/server:2019-RC1
mcr.microsoft.com/mssql/server   2019-RC1             c5a295efea97        7 weeks ago         1.5GB


 ## 3-2.SQL Server2019コンテナ起動（SQL Server 2017で作成したデータファイルをマウント）

In [17]:
%%bash
sudo docker run -e 'ACCEPT_EULA=Y' -e 'MSSQL_SA_PASSWORD=P@ssw0rd' -p 11433:1433 -v ~/SQL_Server_Container/DataVolume:/var/opt/mssql --name sql19rc1 -d mcr.microsoft.com/mssql/server:2019-RC1
sudo docker ps -a --filter name=sql19rc1
sleep 30


28e82df65694f39048d62a22b1d38df0de4d8fb1f1fa411fd3aed9a825d6c608
CONTAINER ID        IMAGE                                     COMMAND                  CREATED             STATUS                  PORTS                     NAMES
28e82df65694        mcr.microsoft.com/mssql/server:2019-RC1   "/opt/mssql/bin/perm…"   2 seconds ago       Up Less than a second   0.0.0.0:11433->1433/tcp   sql19rc1


 ## 3-3.SQL Server に接続すると”Upgrade中だから待て”のエラー
 ### Sqlcmd: Error: Microsoft ODBC Driver 17 for SQL Server : Login failed for user 'sa'. Reason: Server is in script upgrade mode. Only administrator can connect at this time..

In [18]:
%%bash
sudo docker exec sql19rc1 /opt/mssql-tools/bin/sqlcmd -S localhost -U sa -P P@ssw0rd -Q 'select @@version'


Sqlcmd: Error: Microsoft ODBC Driver 17 for SQL Server : Login failed for user 'sa'. Reason: Server is in script upgrade mode. Only administrator can connect at this time..


CalledProcessError: Command 'b"sudo docker exec sql19rc1 /opt/mssql-tools/bin/sqlcmd -S localhost -U sa -P P@ssw0rd -Q 'select @@version'\n"' returned non-zero exit status 1

 ## 3-4.アップグレードが終われば接続できるようになる
 ### Microsoft SQL Server 2019 (RC1) - 15.0.1900.25 (X64)

In [20]:
%%bash
sudo docker exec sql19rc1 /opt/mssql-tools/bin/sqlcmd -S localhost -U sa -P P@ssw0rd -Q 'select @@version'



------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Microsoft SQL Server 2019 (RC1) - 15.0.1900.25 (X64) 
	Aug 16 2019 14:20:53 
	Copyright (C) 2019 Microsoft Corporation
	Developer Edition (64-bit) on Linux (Ubuntu 16.04.6 LTS) <X64>                                                                                                                      

(1 rows affected)


 ## 3-5.SQL Server 2017から2019にアップグレードしたDBをSELECT

In [21]:
%%bash
sqlcmd -S localhost,11433 -U sa -P P@ssw0rd -Q 'use DevDB; select * from tab01'


Changed database context to 'DevDB'.
id          name                                                                                                                                                                                                                                                            
----------- ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
          1 SQL Server runs on Docker!                                                                                                                                                                                                                                      

(1 rows affected)


 # Demo 4: SQL Server コンテナ内のDBバックアップ
 ## 4-1.フルバックアップ

In [22]:
%%bash
sqlcmd -S localhost,11433 -U sa -P P@ssw0rd -Q 'backup database DevDB to disk="/tmp/DevDB.bak" with format'
sudo docker exec sql19rc1 ls -la /tmp/DevDB.bak


Processed 384 pages for database 'DevDB', file 'DevDB' on file 1.
Processed 2 pages for database 'DevDB', file 'DevDB_log' on file 1.
BACKUP DATABASE successfully processed 386 pages in 0.692 seconds (4.352 MB/sec).
-rw-r----- 1 root root 3264512 Oct 10 12:39 /tmp/DevDB.bak


 ## 4-2.バックアップをホストにコピー

In [23]:
%%bash
sudo docker cp sql19rc1:/tmp/DevDB.bak ~/SQL_Server_Container/DevDB.bak
ls -la ~/SQL_Server_Container/DevDB.bak


-rw-r----- 1 root root 3264512 Oct 10 12:39 /home/o9o9/SQL_Server_Container/DevDB.bak


 ## 4-3.DB削除＆DBファイル削除

In [24]:
%%bash
sudo docker rm -f sql19rc1
sudo docker ps -a --filter name=sql19rc1
sudo rm -r ~/SQL_Server_Container/DataVolume/*


sql19rc1
CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES


 # Demo 5: SQL Server カスタムコンテナイメージの作成と配布
 ## 5-1.Dockerfile編集

In [25]:
%%bash
cd ~/SQL_Server_Container
touch Dockerfile
echo 'FROM mcr.microsoft.com/mssql/server:2019-RC1' >> Dockerfile
echo 'COPY ./DevDB.bak /tmp/DevDB.bak' >> Dockerfile
echo 'CMD ["/opt/mssql/bin/sqlservr"]' >> Dockerfile
cat Dockerfile


FROM mcr.microsoft.com/mssql/server:2019-RC1
COPY ./DevDB.bak /tmp/DevDB.bak
CMD ["/opt/mssql/bin/sqlservr"]


 ## 5-2.カスタムコンテナイメージのビルド

In [26]:
%%bash
cd ~/SQL_Server_Container
sudo docker build -t localhost:5000/mssql/devdb:v1 .
sudo docker history localhost:5000/mssql/devdb:v1
sudo docker image list | grep devdb



Step 1/3 : FROM mcr.microsoft.com/mssql/server:2019-RC1
 ---> c5a295efea97
Step 2/3 : COPY ./DevDB.bak /tmp/DevDB.bak
 ---> 215f50b04d5f
Step 3/3 : CMD ["/opt/mssql/bin/sqlservr"]
 ---> Running in e6f943c4f27c
Removing intermediate container e6f943c4f27c
 ---> 846746a17505
Successfully built 846746a17505
Successfully tagged localhost:5000/mssql/devdb:v1
IMAGE               CREATED                  CREATED BY                                      SIZE                COMMENT
846746a17505        Less than a second ago   /bin/sh -c #(nop)  CMD ["/opt/mssql/bin/sqls…   0B                  
215f50b04d5f        2 seconds ago            /bin/sh -c #(nop) COPY file:fc8d3bf5d9c298c0…   3.26MB              
c5a295efea97        7 weeks ago              /bin/sh -c #(nop)  CMD ["/opt/mssql/bin/sqls…   0B                  
<missing>           7 weeks ago              /bin/sh -c #(nop)  ENTRYPOINT ["/opt/mssql/b…   0B                  
<missing>           7 weeks ago              /bin/sh -c /tmp/insta

 ## 5-3.ローカルプライベートレジストリ起動

In [27]:
%%bash
sudo docker run -d -p 5000:5000 --name registry -v ~/SQL_Server_Container/RegistryVolume:/var/lib/registry registry


ee109e5b449524a56872ffc2d2fa7ba5a394fbfcfd4f490a5186339683c23853


 ## 5-4.ローカルプライベートレジストリにプッシュ（事前プッシュ済みなので早い）

In [28]:
%%bash
sudo docker push localhost:5000/mssql/devdb:v1


The push refers to repository [localhost:5000/mssql/devdb]
415673e6868e: Preparing
291d0eb6856c: Preparing
64a2fdccc647: Preparing
25ed665d515e: Preparing
e4e0ec1b4536: Preparing
2de391e51d73: Preparing
d73dd9e65295: Preparing
686245e78935: Preparing
d7ff1dc646ba: Preparing
644879075e24: Preparing
d73dd9e65295: Waiting
686245e78935: Waiting
d7ff1dc646ba: Waiting
644879075e24: Waiting
2de391e51d73: Waiting
291d0eb6856c: Layer already exists
e4e0ec1b4536: Layer already exists
25ed665d515e: Layer already exists
d73dd9e65295: Layer already exists
2de391e51d73: Layer already exists
64a2fdccc647: Layer already exists
686245e78935: Layer already exists
d7ff1dc646ba: Layer already exists
644879075e24: Layer already exists
415673e6868e: Pushed
v1: digest: sha256:65d8788b4ea5652fc7896f37dc709dc5d001e1c47534a78c75fba978f39e6c79 size: 2411


 ## 5-5.ローカルプライベートレジストリ確認

In [30]:
%%bash
wget http://localhost:5000/v2/_catalog -q -O /dev/stdout
wget http://localhost:5000/v2/mssql/devdb/tags/list -q -O /dev/stdout


{"repositories":["mssql/devdb"]}
{"name":"mssql/devdb","tags":["v1"]}


 ## 5-6.ローカルに作成したカスタムコンテナイメージを削除（明示的にプライベートレジストリからPULLするため）

In [31]:
%%bash
sudo docker rmi localhost:5000/mssql/devdb:v1
sudo docker image list


Untagged: localhost:5000/mssql/devdb:v1
Untagged: localhost:5000/mssql/devdb@sha256:65d8788b4ea5652fc7896f37dc709dc5d001e1c47534a78c75fba978f39e6c79
Deleted: sha256:846746a17505fa7a951eb3b0b05922505ff6140b35ed6d92234c631ed2995105
Deleted: sha256:215f50b04d5f91013f49f1e76b620595d9477a86fb86450c98603226003f5c4e
Deleted: sha256:ada624f28269ba22989878f6be1fa1d93b39732e5f0092a0cf7fa863ad75929b
REPOSITORY                       TAG                  IMAGE ID            CREATED             SIZE
hello                            latest               ef87e37c1855        3 weeks ago         64.2MB
mcr.microsoft.com/mssql/server   2019-RC1             c5a295efea97        7 weeks ago         1.5GB
ubuntu                           latest               a2a15febcdf3        8 weeks ago         64.2MB
mcr.microsoft.com/mssql/server   2017-CU16            d9b9b96627b7        2 months ago        1.36GB
mcr.microsoft.com/mssql/ha       2019-CTP3.2-ubuntu   a83ba87d0520        2 months ago        506MB
regist

 ## 5-7.ローカルプライベートレジストリから Pull & Run

In [32]:
%%bash
sudo docker run -e 'ACCEPT_EULA=Y' -e 'MSSQL_SA_PASSWORD=P@ssw0rd' -p 11433:1433 -v ~/SQL_Server_Container/DataVolume:/var/opt/mssql --name sql19rc1 -d localhost:5000/mssql/devdb:v1
sudo docker ps -a --filter name=sql19rc1
sudo docker image list | grep devdb


a14f21f9735222c35c38cf96a689539e6e397582d34f92599b6dfe712c325b6c
CONTAINER ID        IMAGE                           COMMAND                  CREATED             STATUS                  PORTS                     NAMES
a14f21f97352        localhost:5000/mssql/devdb:v1   "/opt/mssql/bin/perm…"   2 seconds ago       Up Less than a second   0.0.0.0:11433->1433/tcp   sql19rc1
localhost:5000/mssql/devdb       v1                   846746a17505        4 minutes ago       1.5GB
Unable to find image 'localhost:5000/mssql/devdb:v1' locally
v1: Pulling from mssql/devdb
b234f539f7a1: Already exists
55172d420b43: Already exists
5ba5bbeb6b91: Already exists
43ae2841ad7a: Already exists
f6c9c6de4190: Already exists
e39f945bda21: Already exists
6689ce95f395: Already exists
ec004dcfdfb5: Already exists
e44708601d04: Already exists
5f1883e23375: Pulling fs layer
5f1883e23375: Verifying Checksum
5f1883e23375: Download complete
5f1883e23375: Pull complete
Digest: sha256:65d8788b4ea5652fc7896f37dc709dc5d001

 ## 5-8.カスタム（バックアップファイルを配置）を確認

In [33]:
%%bash
sudo docker exec sql19rc1 ls -l /tmp


total 3188
-rw-r----- 1 root root 3264512 Oct 10 12:39 DevDB.bak


 # Demo 6: SQL Server カスタムコンテナへのDBリストア
 ## 6-1.SELECT（リストア前）

In [34]:
%%bash
sqlcmd -S localhost,11433 -U sa -P P@ssw0rd -Q 'use DevDB; select * from tab01'


Msg 911, Level 16, State 1, Server a14f21f97352, Line 1
Database 'DevDB' does not exist. Make sure that the name is entered correctly.


 ## 6-2.リストア

In [35]:
%%bash
sqlcmd -S localhost,11433 -U sa -P P@ssw0rd -Q 'RESTORE DATABASE [DevDB] FROM DISK = "/tmp/DevDB.bak"'


Processed 384 pages for database 'DevDB', file 'DevDB' on file 1.
Processed 2 pages for database 'DevDB', file 'DevDB_log' on file 1.
RESTORE DATABASE successfully processed 386 pages in 0.393 seconds (7.663 MB/sec).


 ## 6-3.SELECT

In [36]:
%%bash
sqlcmd -S localhost,11433 -U sa -P P@ssw0rd -Q 'use DevDB; select * from tab01'


Changed database context to 'DevDB'.
id          name                                                                                                                                                                                                                                                            
----------- ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
          1 SQL Server runs on Docker!                                                                                                                                                                                                                                      

(1 rows affected)


 # Demo 7: AutoTuning
 ## 7-1.手動チューニング
 ### 7-1-1.初期化

In [37]:
%%bash
sudo docker restart sqlautotune
sleep 10
sqlcmd -S localhost,61433 -U sa -P P@ssw0rd \
  -i ~/SQL_Server_Container/SQL_Server_Autotune/2.init_autotune_off.sql


sqlautotune
Changed database context to 'demodb'.
DBCC execution completed. If DBCC printed error messages, contact your system administrator.


 ### 7-1-2.スループット収集
 Azure Data Studio で 3.batchrequests_perf_collector.sql を実行
 ![](image/2019-10-09-23-38-20.png)
 ### 7-1-3.ワークロード実行
 Azure Data Studio で 4.report.sql を実行
 ![](image/2019-10-09-23-39-50.png)
 ### 7-1-4.スループット推移確認
 Azure Data Studio で 5.batchrequests.sql を実行
 ![](image/2019-10-09-23-41-41.png)
 ### 7-1-5.パラメータスニッフィング問題発生
 Azure Data Studio で 6.regression.sql を実行
 ![](image/2019-10-09-23-42-59.png)
 ### 7-1-6.スループット推移確認(7-1-4に比較しスローダウンしていることを確認)
 Azure Data Studio で 5.batchrequests.sql を実行
 ![](image/2019-10-09-23-44-53.png)
 ### 7-1-7.チューニングの推奨を確認
 Azure Data Studio で 7.recommendations.sql を実行
 ![](image/2019-10-09-23-45-58.png)
 ### 7-1-8.推奨を手動適用
 Azure Data Studio で 8.manual_tune.sql を実行
 ![](image/2019-10-09-23-47-10.png)
 ### 7-1-9.スループット推移確認(スループットの改善を確認)
 Azure Data Studio で 5.batchrequests.sql を実行
 ![](image/2019-10-09-23-48-57.png)

 ## 7-2.自動チューニング
 ### 7-2-1.初期化

In [38]:
%%bash
sudo docker restart sqlautotune
sleep 10
sqlcmd -S localhost,61433 -U sa -P P@ssw0rd \
  -i ~/SQL_Server_Container/SQL_Server_Autotune/2.init_autotune_on.sql


sqlautotune
Changed database context to 'demodb'.
DBCC execution completed. If DBCC printed error messages, contact your system administrator.


 ### 7-2-2.スループット収集
 Azure Data Studio で 3.batchrequests_perf_collector.sql を実行
 ![](image/2019-10-09-23-38-20.png)
 ### 7-2-3.ワークロード実行
 Azure Data Studio で 4.report.sql を実行
 ![](image/2019-10-09-23-39-50.png)
 ### 7-2-4.スループット推移確認
 Azure Data Studio で 5.batchrequests.sql を実行
 ![](image/2019-10-09-23-56-40.png)
 ### 7-2-5.パラメータスニッフィング問題発生
 Azure Data Studio で 6.regression.sql を実行
 ![](image/2019-10-09-23-42-59.png)
 ### 7-2-6.スループット推移確認(7-2-4に比較しスローダウンしていることを確認)
 Azure Data Studio で 5.batchrequests.sql を実行
 ![](image/2019-10-09-23-58-20.png)
 ### 7-1-7.チューニングの推奨を確認
 Azure Data Studio で 7.recommendations.sql を実行
 ![](image/2019-10-09-23-59-43.png)
 ### 7-2-8.スループット推移確認(スループットの改善を確認)
 Azure Data Studio で 5.batchrequests.sql を実行
 ![](image/2019-10-10-00-17-54.png)
 ### 7-2-9.スループット推移確認([Reverted](https://docs.microsoft.com/ja-jp/sql/relational-databases/system-dynamic-management-views/sys-dm-db-tuning-recommendations-transact-sql?view=sql-server-2017#remarks)となるケースもあり)
 Azure Data Studio で 5.batchrequests.sql を実行
 ![](image/2019-10-10-00-14-19.png)